Author: Xinnan Shen<br>
Date: 01-09-2020

# Learned Index Model Classification Demo
This notebook is to build some simple learned index models based on Kraska's paper. (classification model)

In this model, I have used pages instead of real locations in databases in the dataset.

Step 1: Generate some simple dataset in csv format

In [ ]:
import os
import codecs
import random
#function: data_generation
#usage: generate a simple dataset
#parameters:
#1.len_num: the size of dataset
#2.range_min: the minimum key
#3.range_max: the maximun key
#output dataset: two columns (key,location)
def data_generation(len_num,range_min,range_max):
	dataset=set()
	for i in range(0,len_num):
		x=random.randint(range_min,range_max)
		while x in dataset:
			x=random.randint(range_min,range_max)
		dataset.add(x)
	# for i in range(0,len(datalist)):
	# 	temp=False
	# 	for j in range(0,len(datalist)-i-1):
	# 		if datalist[j]>datalist[j+1]:
	# 			t=datalist[j]
	# 			datalist[j]=datalist[j+1]
	# 			datalist[j+1]=t
	# 			temp=True
	# 	if not temp:
	# 		break
	current_path=os.path.abspath(os.curdir)
	f=codecs.open(os.path.join(current_path,"data.csv"), "w", "utf-8")
	i=0
	datalist=list(dataset)
	datalist.sort()
	for ele in datalist:
		f.write(str(ele)+","+str(int(i/(len_num/100)))+"\n")
		i=i+1
	f.close()
	return


Provide some value and generate the dataset

In [ ]:
minkey=0
maxkey=2147483647
keynum=5000000
data_generation(keynum,minkey,maxkey)

Step 2: Split the dataset into training, development and testing dataset

In [15]:
from random import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
current_path=os.path.abspath(os.curdir)
f=codecs.open(os.path.join(current_path,"data.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
list_key=[]
list_res=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    list_key.append(temp[0])
    list_res.append(temp[1])
keys=np.array(list_key)
res=np.array(list_res)
trainkeys,testkeys,trainres,testres=train_test_split(keys,res,test_size=0.35)
trainkeys,devkeys,trainres,devres=train_test_split(trainkeys,trainres,test_size=0.5)
trainkeys=list(trainkeys)
devkeys=list(devkeys)
testkeys=list(testkeys)
trainres=list(trainres)
devres=list(devres)
testres=list(testres)

f=codecs.open(os.path.join(current_path,"data_train.csv"), "w", "utf-8")
for i in range(0,len(trainkeys)):
    f.write(str(trainkeys[i])+","+str(trainres[i])+"\n")
f.close()
f=codecs.open(os.path.join(current_path,"data_dev.csv"), "w", "utf-8")
for i in range(0,len(devkeys)):
    f.write(str(devkeys[i])+","+str(devres[i])+"\n")
f.close()
f=codecs.open(os.path.join(current_path,"data_test.csv"), "w", "utf-8")
for i in range(0,len(testkeys)):
    f.write(str(testkeys[i])+","+str(testres[i])+"\n")
f.close()
print("training data size:",len(trainkeys))
print("development data size:",len(devkeys))
print("testing data size:",len(testkeys))

FileNotFoundError: ignored

In [23]:
import codecs
import os
minkey=1000
maxkey=9999
keynum=3000
current_path=os.path.abspath(os.curdir)
f=codecs.open(os.path.join(current_path,"data_train.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
trainkeys=[]
trainres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    trainkeys.append(int(temp[0]))
    trainres.append(int(temp[1]))
f=codecs.open(os.path.join(current_path,"data_dev.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
devkeys=[]
devres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    devkeys.append(int(temp[0]))
    devres.append(int(temp[1]))
f=codecs.open(os.path.join(current_path,"data_test.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
testkeys=[]
testres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    testkeys.append(int(temp[0]))
    testres.append(int(temp[1]))
print("training data size:",len(trainkeys))
print("development data size:",len(devkeys))
print("testing data size:",len(testkeys))

training data size: 1625000
development data size: 1625000
testing data size: 1750000


In [24]:
import numpy as np
X_train=np.array(trainkeys).reshape(-1,1)
Y_train=np.array(trainres).reshape(-1,1)
X_dev=np.array(devkeys).reshape(-1,1)
Y_dev=np.array(devres).reshape(-1,1)
X_test=np.array(testkeys).reshape(-1,1)

Step 4: Build a model

- KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
import time
import numpy as np
from sklearn.metrics import classification_report
t1=time.time()
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(X_train,Y_train)
t2=time.time()
time_interval=t2-t1
devpre=neigh.predict(X_dev)#.reshape(1,-1).tolist()[0]
print(classification_report(Y_dev,devpre))
print("time interval for building model:"+str(time_interval*1000)+" ms")
t1=time.time()
testpre=neigh.predict(X_test)#.reshape(1,-1).tolist()[0]
t2=time.time()
time_interval=t2-t1
print("time interval for indexing data:"+str(time_interval*1000)+" ms")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16251
           1       1.00      1.00      1.00     16224
           2       1.00      1.00      1.00     16227
           3       1.00      1.00      1.00     16269
           4       1.00      1.00      1.00     16354
           5       1.00      1.00      1.00     16234
           6       1.00      1.00      1.00     16364
           7       1.00      1.00      1.00     16104
           8       1.00      1.00      1.00     16316
           9       1.00      1.00      1.00     16212
          10       1.00      1.00      1.00     16330
          11       1.00      1.00      1.00     16042
          12       1.00      1.00      1.00     16258
          13       1.00      1.00      1.00     16194
          14       1.00      1.00      1.00     16213
          15       1.00      1.00      1.00     16316
          16       1.00      1.00      1.00     16430
          17       1.00    

- Naive Bayes

In [27]:
from sklearn.naive_bayes import GaussianNB
import time
import numpy as np
from sklearn.metrics import classification_report
t1=time.time()
NB = GaussianNB()
NB.fit(X_train,Y_train)
t2=time.time()
time_interval=t2-t1
devpre=NB.predict(X_dev)#.reshape(1,-1).tolist()[0]
print(classification_report(Y_dev,devpre))
print("time interval for building model:"+str(time_interval*1000)+" ms")
t1=time.time()
testpre=NB.predict(X_test)#.reshape(1,-1).tolist()[0]
t2=time.time()
time_interval=t2-t1
print("time interval for indexing data:"+str(time_interval*1000)+" ms")

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16251
           1       1.00      1.00      1.00     16224
           2       1.00      1.00      1.00     16227
           3       1.00      1.00      1.00     16269
           4       1.00      1.00      1.00     16354
           5       1.00      1.00      1.00     16234
           6       1.00      1.00      1.00     16364
           7       1.00      1.00      1.00     16104
           8       1.00      1.00      1.00     16316
           9       1.00      1.00      1.00     16212
          10       1.00      1.00      1.00     16330
          11       1.00      1.00      1.00     16042
          12       0.99      1.00      1.00     16258
          13       1.00      0.99      1.00     16194
          14       1.00      1.00      1.00     16213
          15       1.00      1.00      1.00     16316
          16       1.00      1.00      1.00     16430
          17       1.00    

- Decision Tree

In [28]:
from sklearn import tree
import time
import numpy as np
from sklearn.metrics import classification_report
t1=time.time()
tree = tree.DecisionTreeClassifier()
tree.fit(X_train,Y_train)
t2=time.time()
time_interval=t2-t1
devpre=tree.predict(X_dev)#.reshape(1,-1).tolist()[0]
print(classification_report(Y_dev,devpre))
print("time interval for building model:"+str(time_interval*1000)+" ms")
t1=time.time()
testpre=tree.predict(X_test)#.reshape(1,-1).tolist()[0]
t2=time.time()
time_interval=t2-t1
print("time interval for indexing data:"+str(time_interval*1000)+" ms")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16251
           1       1.00      1.00      1.00     16224
           2       1.00      1.00      1.00     16227
           3       1.00      1.00      1.00     16269
           4       1.00      1.00      1.00     16354
           5       1.00      1.00      1.00     16234
           6       1.00      1.00      1.00     16364
           7       1.00      1.00      1.00     16104
           8       1.00      1.00      1.00     16316
           9       1.00      1.00      1.00     16212
          10       1.00      1.00      1.00     16330
          11       1.00      1.00      1.00     16042
          12       1.00      1.00      1.00     16258
          13       1.00      1.00      1.00     16194
          14       1.00      1.00      1.00     16213
          15       1.00      1.00      1.00     16316
          16       1.00      1.00      1.00     16430
          17       1.00    

- Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier
import time
import numpy as np
from sklearn.metrics import classification_report
t1=time.time()
forest = RandomForestClassifier()
forest.fit(X_train,Y_train)
t2=time.time()
time_interval=t2-t1
devpre=forest.predict(X_dev)#.reshape(1,-1).tolist()[0]
print(classification_report(Y_dev,devpre))
print("time interval for building model:"+str(time_interval*1000)+" ms")
t1=time.time()
testpre=forest.predict(X_test)#.reshape(1,-1).tolist()[0]
t2=time.time()
time_interval=t2-t1
print("time interval for indexing data:"+str(time_interval*1000)+" ms")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16251
           1       1.00      1.00      1.00     16224
           2       1.00      1.00      1.00     16227
           3       1.00      1.00      1.00     16269
           4       1.00      1.00      1.00     16354
           5       1.00      1.00      1.00     16234
           6       1.00      1.00      1.00     16364
           7       1.00      1.00      1.00     16104
           8       1.00      1.00      1.00     16316
           9       1.00      1.00      1.00     16212
          10       1.00      1.00      1.00     16330
          11       1.00      1.00      1.00     16042
          12       1.00      1.00      1.00     16258
          13       1.00      1.00      1.00     16194
          14       1.00      1.00      1.00     16213
          15       1.00      1.00      1.00     16316
          16       1.00      1.00      1.00     16430
          17       1.00    

- DNN

In [ ]:
from keras.utils.np_utils import to_categorical   
Y_train_NN = to_categorical(Y_train, num_classes=100)
Y_dev_NN = to_categorical(Y_dev, num_classes=100)

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api
from keras import models as md
from keras import layers as lr
import numpy as np
import time
t1=time.time()
model = md.Sequential()
model.add(lr.Dense(128,activation="relu"))
model.add(lr.Dense(128,activation="relu"))
# model.add(lr.Dense(32,activation="relu"))
model.add(lr.Dropout(0.2))
model.add(lr.Dense(100,activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])#compile the model
model.fit(X_train, Y_train_NN, epochs=16, batch_size=32)#fit the model
t2=time.time()
time_interval=t2-t1
loss, acc = model.evaluate(X_dev, Y_dev_NN)
print(loss,acc)
print("time interval for building model:"+str(time_interval*1000)+" ms")
t1=time.time()
model.predict(X_test)
t2=time.time()
time_interval=t2-t1
print("time interval for indexing data:"+str(time_interval*1000)+" ms")

Epoch 1/16
2032/2032 [==============================] - 5s 2ms/step - loss: 1401627.1250 - accuracy: 0.0097
Epoch 2/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6055 - accuracy: 0.0096
Epoch 3/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6055 - accuracy: 0.0101
Epoch 4/16
2032/2032 [==============================] - 5s 3ms/step - loss: 4.6054 - accuracy: 0.0101
Epoch 5/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6055 - accuracy: 0.0095
Epoch 6/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6054 - accuracy: 0.0100
Epoch 7/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6054 - accuracy: 0.0096
Epoch 8/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6054 - accuracy: 0.0097
Epoch 9/16
2032/2032 [==============================] - 5s 2ms/step - loss: 4.6054 - accuracy: 0.0095
Epoch 10/16
2032/2032 [==============================] - 5s 3ms/step - loss:

In [ ]:
Y_test=np.argmax(model.predict(X_test), axis=-1)
print(Y_test)

[70 70 70 ... 70 70 70]
